In [90]:
from tensorflow.keras.models import load_model
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt 

CATEGORIES_REV = {
    0: 'alert', 1: 'button', 2: 'card', 3:'checkbox_checked',
    4: 'checkbox_unchecked', 5: 'chip', 6: 'data_table', 7:'dropdown_menu',
    8: 'floating_action_button', 9: 'grid_list', 10: 'image', 11: 'label', 12: 'menu',
    13: 'radio_button_checked', 14: 'radio_button_unchecked', 15: 'slider', 16: 'switch_disabled',
    17: 'switch_enabled', 18: 'text_area', 19: 'text_field', 20: 'tooltip'
}

In [93]:
class PredictWithModel:
    shape = (224, 224, 1)
    
    def __init__(self, _model_path: str, _test_dir: str='data/test') -> None:
        self.model = load_model(_model_path)
        
        self.paths = os.listdir(_test_dir)
        self.images = np.empty((_s:=len(self.paths), ) + self.shape, dtype='float16')
        
        self.test_dir = _test_dir
        self.predictions = pd.DataFrame(columns=['ID', 'Target'])
        
        self.__index, self.size = 0, _s 
    
    def test_img_from_dir(self) -> None:
        for img_path in self.paths:
            self.images[self.__index] = self._parse_img(img_path)
            self.__index += 1
            
            #print(f"[{self.__index}/{self.images.shape[0]}] {img_path}")
            
    @staticmethod
    def show(img: np.array) -> None:
        plt.imshow(img[0], cmap='gray')
    
    def _parse_img(self, _img_path: str) -> np.array:
        img = ~cv2.imread(f"{self.test_dir}/{_img_path}", 0) / 255
        
        return np.asarray(img, dtype='float16').reshape(self.shape)
    
    def predict_img(self):
        self.test_img_from_dir()
        
        for i, p in enumerate(self.model.predict(self.images)):
            self.predictions.loc[i] = [self.paths[i], CATEGORIES_REV[np.argmax(p)]]
        
        self.predictions.to_csv('data/result/predictions.csv', index=False)


In [94]:
pwm = PredictWithModel('models/model_v0.2.1.h5')

pwm.predict_img()

161/161 [==============================] - 16s 100ms/step
